# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

In [76]:
import fullcontrol as fc
import math

In [77]:
# printer/gcode parameters

design_name = 'armband'
nozzle_temp = 210
bed_temp = 60
print_speed = 400
fan_percent = 100
printer_name='prusa_i3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [78]:
# design parameters

EW = 0.4 # extrusion width
EH = 0.3 # extrusion height (and layer height)
initial_z = EH*0.3 # initial nozzle position is set to 0.6x the extrusion height to get a bit of 'squish' for good bed adhesion

total_height = 3 # [mm]
circumference = 60 # [mm]
num_layer = 2
squish_factor = 0.3 # squish lines in % of extrusion width
overlap_factor = 0.3 # overlap in % between lines
center_point = (40, 40) # [mm]

In [79]:
def generate_circle_points(num_points, radius, phase=0, offset_x=0, offset_y=0):
    circle_points = []
    for i in range(num_points):
        theta = 2 * math.pi * i / num_points + phase
        x = radius * math.cos(theta) + offset_x
        y = radius * math.sin(theta) + offset_y
        circle_points.append((x, y))
    return circle_points

In [80]:
# generate the design (make sure you've run the above cells before running this cell)

steps = []

num_lines = int(circumference / (EW * (1 - squish_factor)) / 2)
layer_height = total_height / num_layer
for layer in range(num_layer):
  radius = circumference/(2*math.pi)
  phase = (layer % 2) * 2 * math.pi/circumference * EW
  circle_points = generate_circle_points(num_lines, radius, phase, *center_point)
  # steps.append(fc.Extruder(on=False))
  steps.append(fc.Extruder(on=True))
  for pt in circle_points:

    steps.append(fc.Point(x=pt[0], y=pt[1], z=initial_z + max(0, layer_height * (layer - overlap_factor))))
    # steps.append(fc.Extruder(on=True))
    steps.append(fc.Point(x=pt[0]+0.01, y=pt[1]+0.01, z=initial_z + min(total_height, layer_height * (layer + overlap_factor + 1))))
    # steps.append(fc.Extruder(on=False))
    steps.append(fc.StationaryExtrusion(volume=-0.2, speed=60)) # retraction


In [81]:
# preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
# fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [82]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    save_as=design_name,
    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109. 
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol